pip install tensorflow  


In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import imageio
import math

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [123]:
patch_size = 64
batch_size = 10
percent_used_for_validation = 0.15

In [117]:
def make_nn():
    L = 9
    kernel_size = 5
    input_channels = 40
    hidden_channels = 100
    in_between_layers = 7
    # output_kernel_size = 28

    model = keras.Sequential()
    model.add(keras.Input(shape=(patch_size, patch_size, input_channels)))
    for i in range(in_between_layers):
        model.add(keras.layers.ZeroPadding2D(padding=(2, 2)))
        model.add(layers.Conv2D(hidden_channels, kernel_size=kernel_size, strides=1, activation="relu"))
    model.add(keras.layers.ZeroPadding2D(padding=(2, 2)))
    # model.add(layers.Conv2D(output_kernel_size**2, kernel_size=kernel_size, strides=1, activation="relu"))
    model.add(layers.Conv2D(3, kernel_size=kernel_size, strides=1, activation="relu"))
    model.compile(loss="mean_absolute_error", optimizer="adam", metrics=['accuracy'])
    return model;
model = make_nn()

model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_42 (ZeroPaddi (None, 68, 68, 40)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 64, 64, 100)       100100    
_________________________________________________________________
zero_padding2d_43 (ZeroPaddi (None, 68, 68, 100)       0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 64, 64, 100)       250100    
_________________________________________________________________
zero_padding2d_44 (ZeroPaddi (None, 68, 68, 100)       0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 64, 64, 100)       250100    
_________________________________________________________________
zero_padding2d_45 (ZeroPaddi (None, 68, 68, 100)      

In [132]:
def preprocess_color(color, albedo, epsilon=0.00316):
    return color * (albedo + epsilon)

def calculate_gradient(image):
    y, x, c = image.shape
    dx = image[:, 1:, :] - image[:, :x-1, :]
    dy = image[1:, :, :] - image[:y-1, :, :]
    dx = np.append(dx, np.zeros([y, 1, c]), axis=1)
    dy = np.append(dy, np.zeros([1, x, c]), axis=0)
    grad = np.dstack((dx, dy))
    return grad

def load_highspp(folder_path):
    albedo = imageio.imread(folder_path + "/albedo.png")
    color = imageio.imread(folder_path + "/color.png")
    color = preprocess_color(color, albedo)
    return color

def load_lowspp(folder_path):
    # albedo buffers
    albedo = imageio.imread(folder_path + "/albedo.png")
    albedo_variance = imageio.imread(folder_path + "/albedoVariance.png")
    albedo_gradient = calculate_gradient(albedo)

    # color buffers
    color = imageio.imread(folder_path + "/color.png")
    color = preprocess_color(color, albedo)
    color_variance = imageio.imread(folder_path + "/colorVariance.png")
    color_gradient = calculate_gradient(color)

    # depth buffers
    depth = imageio.imread(folder_path + "/depth.png")
    depth = depth.reshape((depth.shape[0], depth.shape[1], 1))
    depth_variance = imageio.imread(folder_path + "/depthVariance.png")
    depth_gradient = calculate_gradient(depth)

    # normal buffers
    normal = imageio.imread(folder_path + "/normal.png")
    normal_variance = imageio.imread(folder_path + "/normalVariance.png")
    normal_gradient = calculate_gradient(normal)


    combined = np.dstack((albedo, albedo_variance, albedo_gradient, color, color_variance, color_gradient, depth, depth_variance, depth_gradient, normal, normal_variance, normal_gradient))
    return combined

In [133]:
# split full res image into patch_size * patch_size images
def get_patches(image):
    x_patches = math.floor(image.shape[0]/patch_size)
    y_patches = math.floor(image.shape[1]/patch_size)
    patches = []
    for x in range(x_patches):
        for y in range(y_patches):
            xstart = x * patch_size
            xend = xstart + patch_size
            ystart = y * patch_size
            yend = ystart + patch_size
            temp = image[xstart:xend, ystart:yend, :]
            patches.append(temp)
    return patches

In [157]:
def prepare_training_data():
    x_patches = []
    y_patches = []
    orig = next(os.walk('./pngs'))[1]
    folders = [x[0:x.rindex('16')] for x in orig if '16' in x]

    for x in folders:
        low_spp = load_lowspp('./pngs/' + x + '16')
        high_spp = load_reference('./pngs/' + x + '4096')
        residual = high_spp - low_spp[:, :, 12:15]
        x_patches = x_patches + get_patches(low_spp)
        y_patches = y_patches + get_patches(residual)
    
    validate_amount = math.floor(len(x_patches) * percent_used_for_validation)
    rec_count = len(x_patches)
    ds = tf.data.Dataset.from_tensor_slices((x_patches, y_patches))
    ds = ds.shuffle(buffer_size=rec_count)
    train = ds.skip(validate_amount).batch(batch_size)
    validate = ds.take(validate_amount).batch(batch_size)
    
    return train, validate

In [156]:
# train, validate = prepare_training_data()
prepare_training_data()

['bathroom2-', 'bathroom', 'bedroom-', 'car-', 'car2-', 'classroom-', 'coffee-', 'cornell-box-', 'curly-hair-', 'dining-room-', 'dragon-', 'furball-', 'glass-of-water-', 'house-', 'kitchen-', 'lamp-', 'living-room-2-', 'living-room-3-', 'living-room-', 'material-testball-', 'spaceship-', 'staircase-', 'staircase2-', 'teapot-', 'teapot-full-', 'veach-ajar-', 'veach-bidir-', 'veach-mis-', 'volumetric-caustic-', 'water-caustic-']


In [136]:
print("Fit model on training data")
history = model.fit(
    train,
    validation_data=validate,
    batch_size=batch_size,
    epochs=10
)

Fit model on training data
Epoch 1/10
19/19 [==============================] - 5s 280ms/step - loss: 3631.1216 - accuracy: 0.6040 - val_loss: 3977.7505 - val_accuracy: 0.5618
Epoch 2/10
19/19 [==============================] - 4s 129ms/step - loss: 3727.0281 - accuracy: 0.5874 - val_loss: 3682.0820 - val_accuracy: 0.5924
Epoch 3/10
19/19 [==============================] - 2s 123ms/step - loss: 3678.7551 - accuracy: 0.6059 - val_loss: 2965.6638 - val_accuracy: 0.5606
Epoch 4/10
19/19 [==============================] - 2s 126ms/step - loss: 4019.6455 - accuracy: 0.5949 - val_loss: 5245.5498 - val_accuracy: 0.5845
Epoch 5/10
19/19 [==============================] - 4s 205ms/step - loss: 3706.0715 - accuracy: 0.6028 - val_loss: 3256.2471 - val_accuracy: 0.6152
Epoch 6/10
19/19 [==============================] - 2s 128ms/step - loss: 3999.0996 - accuracy: 0.6048 - val_loss: 2484.3816 - val_accuracy: 0.6212
Epoch 7/10
19/19 [==============================] - 2s 123ms/step - loss: 3725.5701 -